## This notebook explores the sizes of objects

This needs a bit of fix-up to get a data-sample that we can join with the input parquet file

#### Standard imports

In [ ]:
import tables_io
import numpy as np
import matplotlib.pyplot as plt
import qp
import pyccl as ccl

#### Change this to match your root data area

In [ ]:
# pz_dir = '/global/cfs/cdirs/lsst/groups/PZ/DP1'
# if that fails you can use this
# pz_dir = '/global/u2/e/echarles/dx'
pz_dir = '/Users/echarles/pz'

In [ ]:
t = tables_io.read(f'{pz_dir}/data/dp1_v29.0.0/5063/object.parquet')

#### Set up the cosmology & evaluation grid

In [ ]:
cosmo = ccl.Cosmology(
    Omega_c=0.25, Omega_b=0.05,
    h=0.7, n_s=0.95, sigma8=0.8,
    transfer_function='bbks'
)
z_grid = np.linspace(0., 15., 301)[1:]
a_grid = (1/(1+z_grid))
D_a = cosmo.angular_diameter_distance(a_grid)
rad_to_asec = 360*60*60/(2*np.pi) 
ang_scale = D_a * 1000/ rad_to_asec
dist_mod = cosmo.distance_modulus(a_grid)

In [ ]:
_ = plt.plot(z_grid, D_a) 

In [ ]:
t_gold = tables_io.read(f'{pz_dir}/data/on_sky/dp1_v29.0.0_gold_all.hdf5')
#t_in = tables_io.read(f'{pz_dir}/data/dp1_v29.0.0_gold/5063/object.pq')
t_in = tables_io.read(f'{pz_dir}/data/test/dp1_matched_v4_test.hdf5')

In [ ]:
#qp_ens = qp.read(f'{pz_dir}/projects/dp1/data/gold_baseline/dp1_all/output_estimate_bpz.hdf5')
qp_ens = qp.read(f'{pz_dir}/projects/dp1_v4/data/gold_dp1_6band/output_estimate_bpz.hdf5')

In [ ]:
import pandas as pd

In [ ]:
t_gold_pd = pd.DataFrame(t_gold)
#t_in_pd = t_in.to_pandas()
t_in_pd = pd.DataFrame(t_in)

In [ ]:
def r_95_of_n(n):
    return 1.25 + 1.125*n

In [ ]:
print(t_gold_pd['objectId'].min(),t_gold_pd['objectId'].max())
print(t_in_pd['objectId'].min(),t_in_pd['objectId'].max())

In [ ]:
t = t_gold_pd.join(t_in_pd, on='objectId', how='inner', lsuffix='a', rsuffix='b')

In [ ]:
len(t)

In [ ]:
blendedness = t['i_blendedness'].to_numpy()
blendedness = np.where(np.isnan(blendedness), 0., blendedness).clip(0., 1.)
mask  = np.bitwise_and(
    np.bitwise_and(
        t['i_extendedness'].to_numpy() == 1,
        t['g_psfFlux'].to_numpy() / t['g_psfFluxErr'].to_numpy() > 10,
    ),
    np.bitwise_and(
        t['i_psfFlux'].to_numpy() / t['i_psfFluxErr'].to_numpy() > 10,
        t['r_psfFlux'].to_numpy() / t['r_psfFluxErr'].to_numpy() > 10,
    ),
)
mask_unblend = np.bitwise_and(mask, blendedness<0.1)
mask_blend = np.bitwise_and(mask, blendedness>0.1)
blend = t['detect_fromBlend'].to_numpy()
psf_flux = t['i_psfFlux'].to_numpy()
cModel_flux = t['i_cModelFlux'].to_numpy()
cModel_flux = t['i_sersicFlux'].to_numpy()
shape_xx = t['i_ixx'].to_numpy()
shape_yy = t['i_iyy'].to_numpy()
shape_xy = t['i_ixy'].to_numpy()
psf_xx = t['i_ixxDebiasedPSF'].to_numpy()
psf_yy = t['i_iyyDebiasedPSF'].to_numpy()
psf_xy = t['i_ixyDebiasedPSF'].to_numpy()
shape_det = shape_xx*shape_yy - shape_xy*shape_xy
shape_trace = shape_xx + shape_yy
shape_area = np.sqrt(shape_det)
psf_det = psf_xx*psf_yy - psf_xy*psf_xy
psf_trace = psf_xx + psf_yy
sersic_x = t['sersic_reff_x'].to_numpy()
sersic_y = t['sersic_reff_y'].to_numpy()
sersic_rho = t['sersic_rho'].to_numpy()
sersic_index = t['sersic_index'].to_numpy()
sersic_factor = r_95_of_n(sersic_index)
sersic_xx = sersic_x*sersic_x
sersic_yy = sersic_y*sersic_y
sersic_trace = sersic_xx + sersic_yy
sersic_xy = sersic_x*sersic_y*sersic_rho
sersic_det = sersic_xx*sersic_yy - sersic_xy*sersic_xy
bdReB = t['i_bdReB'].to_numpy()
bdReD = t['i_bdReD'].to_numpy()
kron_rad = t['i_kronRad'].to_numpy()
psf_mag = -2.5 * np.log10(t['i_psfFlux'].to_numpy()) + 31.4
cModel_mag = -2.5 * np.log10(t['i_cModelFlux'].to_numpy()) + 31.4
sersic_mag = -2.5 * np.log10(t['i_sersicFlux'].to_numpy()) + 31.4

In [ ]:
z = qp_ens.ancil['zmode']
a_scale = 1./(1+z)

In [ ]:
D_a = cosmo.angular_diameter_distance(a_scale)
rad_to_asec = 360*60*60/(2*np.pi) 
ang_scale = D_a * 1000/ rad_to_asec

In [ ]:
int_size = np.sqrt(sersic_trace)*0.2 / ang_scale

In [ ]:
blendedness = t['i_blendedness']

In [ ]:
psf_flux